In [ ]:
from fretbursts import *
import phconvert as phc
print('phconvert: ', phc.__version__)

In [ ]:
sns = init_notebook(fs=13, savefig_dpi=75)

In [ ]:
sns.palplot(sns.colors)

In [ ]:
figure_size = (5, 4)
default_figure = lambda: plt.subplots(figsize=figure_size)
save_figures = False

In [ ]:
def savefig(filename, **kwargs):
    if not save_figures:
        return
    import os
    name, extension = os.path.splitext(os.path.basename(filename))
    dir_ = 'figures/%s/' % name
    kwargs_ = dict(dpi=150, bbox_inches='tight', 
                   frameon=True, facecolor='white', transparent=False)
    kwargs_.update(kwargs)
    plt.savefig(dir_ + name + extension, **kwargs_)

# ALEX

In [ ]:
url = 'http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5'
download_file(url, save_dir='./data')

In [ ]:
file_name = './data/' + url.split('/')[-1]
file_name

In [ ]:
phc.hdf5._check_version(file_name)

In [ ]:
d = loader.photon_hdf5(file_name)

In [ ]:
bpl.plot_alternation_hist(d)
loader.usalex_apply_period(d)

In [ ]:
d.calc_bg(bg.exp_fit, time_s=20, tail_min_us='auto', F_bg=1.7)

In [ ]:
dplot(d, timetrace_bg)

In [ ]:
fig, ax = default_figure()

dplot(d, hist_bg_single, tmax=20e-3, ph_sel=Ph_sel(Dex='Aem'), ax=ax,
      plot_style=dict(alpha=0.8),
      fit_style=dict(color='k'))

plt.legend(['Experim. distrib.', 'Exponential fit'])
plt.xlim(0, 15)
plt.xlabel(u'Timestamp waiting time (ms)')
plt.ylabel("# Waiting times")
#plt.title('')
#savefig('ph_delays_distrib1.png')

In [ ]:
fig, ax = default_figure()

dplot(d, hist_bg, ax=ax)

plt.legend(bbox_to_anchor=(1.04, 0.9), loc=2, fontsize=12)
#plt.title('')
savefig('ph_delays_distrib_all.png')

In [ ]:
d.burst_search()

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=15)
ds2 = ds.select_bursts(select_bursts.naa, th1=15)

In [ ]:
sns.palplot(sns.color_palette('RdYlBu_r', 12))

In [ ]:
for cmap in ['YlGnBu_r', 'RdYlBu_r', 'Spectral_r', 'viridis']:
    plt.figure()
    c = sns.color_palette(cmap, 12)[1]
    g = alex_jointplot(ds, cmap=cmap,
                       marginal_kws=dict(hist_bar_style=dict(facecolor=c, edgecolor='k', linewidth=0.2)));
    g.ax_joint.text(-0.1, 1.1, cmap)
    savefig

In [ ]:
cmap = 'RdYlBu_r' #'Spectral_r'
c = sns.color_palette(cmap, 12)[1]
kwargs = dict(cmap=cmap, 
              marginal_kws=dict(hist_bar_style=dict(facecolor=c, edgecolor='k', linewidth=0.2)))
alex_jointplot(ds, **kwargs)
alex_jointplot(ds2,**kwargs)

In [ ]:
# nb_dir = '/Users/anto/Google Drive/notebooks/'
# file_name = nb_dir + 'multispot/8spot_paper/data/2012-11-26/004_dsDNA_17d_green100u_red40u.hdf5'

# d = loader.photon_hdf5(file_name)

# plot_alternation_hist(d)

# loader.alex_apply_period(d)

In [ ]:
import os
nb_dir = '/Users/anto/Google Drive/notebooks/'
data_dir = nb_dir + 'multispot/8spot_paper/data/2012-11-26/'

data_dir = os.path.abspath(data_dir) + '/'
assert os.path.exists(data_dir), "Path '%s' does not exist." % data_dir

from glob import glob
file_list = sorted(f for f in glob(data_dir + '*.hdf5') if '_BKG' not in f)
file_list

In [ ]:
## Selection for POLIMI 2012-11-26 datatset
labels = ['17d', '27d', '7d', '12d', '22d']
files_dict = {lab: fname for lab, fname in zip(labels, file_list)}

file_name = files_dict['27d']
file_name

In [ ]:
phc.hdf5._check_version(files_dict['17d'])

In [ ]:
dd = {k: loader.photon_hdf5(v) for k, v in files_dict.items()}
for d in dd.values():
    loader.usalex_apply_period(d)
    d.calc_bg(bg.exp_fit, time_s=30, tail_min_us='auto', F_bg=1.7)

In [ ]:
dd.keys()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(figure_size[0]*2, figure_size[1]), sharey=True)

dplot(d, timetrace_bg, ax=ax[0])
dplot(dd['17d'], timetrace_bg, ax=ax[1])

plt.legend(['All-ph', 'DexDem', 'DexAem', 'AexAem'],
           #bbox_to_anchor=(1.04, 0.9), 
           ncol=2, loc='upper left', fontsize=12)

ax[0].legend([])
ax[0].set_ylim(0, 5)
ax[1].set_ylabel('')
ax[0].set_title('(a)')
ax[1].set_title('(b)')
savefig('background_timetrace')

In [ ]:
def hist_bg_single(d, i=0, period=0, binwidth=1e-4, bins=None, tmax=0.01,
                   ph_sel=Ph_sel('all'), show_fit=True, yscale='log',
                   manual_rate=None, manual_tau_th=500,
                   xscale='linear', plot_style=None, fit_style=None):
    """Plot histogram of photon interval for a single photon streams.

    Optionally plots the fitted background.
    """

    # If `bins` is not passed or is a scalar create the `bins` array
    if bins is None:
        bins = np.arange(0, tmax + binwidth, binwidth)
    elif np.size(bins) == 1:
        warnings.warn('`bins` is a scalar, `tmax` will be ignored.')
        bins = np.arange(0, bins*binwidth, binwidth)
    t_ax = bins[:-1] + 0.5*binwidth

    # Compute histograms
    ph_times_period = d.get_ph_times_period(ich=i, period=period,
                                            ph_sel=ph_sel)
    delta_ph_t_period = np.diff(ph_times_period)*d.clk_p
    counts, _ = np.histogram(delta_ph_t_period, bins=bins)

    # Max index with counts > 0
    n_trim = np.trim_zeros(counts).size + 1

    # Plot histograms
    plot_style_ = dict(marker='o', markersize=5, linestyle='none', alpha=0.6)
    if ph_sel in bpl._ph_sel_color_dict:
        plot_style_['color'] = bpl._ph_sel_color_dict[ph_sel]
        plot_style_['label'] = repr(ph_sel)
    plot_style_.update(bpl._normalize_kwargs(plot_style, kind='line2d'))
    plot(t_ax[:n_trim]*1e3, counts[:n_trim], **plot_style_)

    if show_fit or manual_rate is not None:
        # Compute the fit function
        if manual_rate is not None:
            bg_rate = manual_rate
            tau_th = manual_tau_th*1e-6
        else:
            bg_rate = d.bg_from(ph_sel)[i][period]
            tau_th = d.bg_th_us[ph_sel][i]*1e-6

        i_tau_th = np.searchsorted(t_ax, tau_th)
        counts_integral = counts[i_tau_th:].sum()
        y_fit = np.exp(- t_ax * bg_rate)
        y_fit *= counts_integral/y_fit[i_tau_th:].sum()

        # Plot
        fit_style_ = dict(plot_style_)
        fit_style_.update(linestyle='-', marker='', label='auto')
        fit_style_.update(bpl._normalize_kwargs(fit_style, kind='line2d'))
        if fit_style_['label'] is 'auto':
            fit_style_['label'] = '%s, %.2f kcps' % (plot_style_['label'],
                                                     bg_rate*1e-3)
        plot(t_ax[:n_trim]*1e3, y_fit[:n_trim], **fit_style_)

    if yscale == 'log':
        gca().set_yscale(yscale)
        plt.ylim(1)
        bpl._plot_status['hist_bg_single'] = {'autoscale': False}
    if xscale == 'log':
        gca().set_xscale(yscale)
        plt.xlim(0.5*binwidth)
        bpl._plot_status['hist_bg_single'] = {'autoscale': False}
    plt.xlabel('Inter-photon delay (ms)')
    plt.ylabel("counts")

def hist_bg(d, i=0, period=0, binwidth=1e-4, bins=None, tmax=0.01,
            show_da=False, show_fit=True, yscale='log', xscale='linear',
            plot_style=None, fit_style=None, legend=True):
    """Plot histogram of photon interval for different photon streams.

    Optionally plots the fitted background.
    """
    # Plot multiple timetraces
    ph_sel_list = [Ph_sel('all'), Ph_sel(Dex='Dem'), Ph_sel(Dex='Aem')]
    if d.ALEX:
         ph_sel_list.append(Ph_sel(Aex='Aem'))
         if show_da:
             ph_sel_list.append(Ph_sel(Aex='Dem'))

    for ix, ph_sel in enumerate(ph_sel_list):
        if not bl.mask_empty(d.get_ph_mask(i, ph_sel=ph_sel)):
            hist_bg_single(d, i=i, period=period, binwidth=binwidth,
                           bins=bins, tmax=tmax, ph_sel=ph_sel,
                           show_fit=show_fit, yscale=yscale, xscale=xscale,
                           plot_style=plot_style, fit_style=fit_style)
    if legend:
        plt.legend(loc='best', fancybox=True)

    if yscale == 'log' or xscale == 'log':
        bpl._plot_status['hist_bg'] = {'autoscale': False}

In [ ]:
fig, ax = default_figure()

dplot(dd['12d'], hist_bg_single, 
      tmax=20e-3, binwidth=1e-4, ph_sel=Ph_sel(Dex='Aem'),
      plot_style=dict(alpha=0.8),
      fit_style=dict(color='k'), ax=ax)

plt.legend(['Experim. distrib.', 'Exponential fit'])

plt.xlabel(u'Timestamp waiting time (ms)')
plt.ylabel("# Waiting times")
#plt.xscale('log')
plt.xlim(1e-1, 10)
#plt.title('')
savefig('ph_delays_distrib1.png')

In [ ]:
repr(Ph_sel('all'))

In [ ]:
d.bg

In [ ]:
fig, ax = default_figure()

dplot(dd['12d'], hist_bg, tmax=0.05, binwidth=2e-4, ax=ax, plot_style=dict(label=None))

plt.legend(bbox_to_anchor=(1.04, 0.9), loc=2, fontsize=12)
plt.xlim(0,15)
#plt.title('')
savefig('ph_delays_distrib_all.png')

In [ ]:
d.burst_search(F=6, m=10, ph_sel=Ph_sel('all'))

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=30, add_naa=True)

In [ ]:
dsh = ds.select_bursts(select_bursts.S, S1=0.25, S2=0.75)

In [ ]:
ds1 = d.select_bursts(select_bursts.size, th1=15)

In [ ]:
ds2 = ds1.select_bursts(select_bursts.naa, th1=15)

In [ ]:
alex_jointplot(ds)
#savefig('alex_jointplot')

In [ ]:
alex_jointplot(dsh)
savefig('alex_jointplot')

In [ ]:
alex_jointplot(ds1)
#savefig('alex_jointplot_noAO')

In [ ]:
alex_jointplot(ds2)
savefig('alex_jointplot_fretsel')

# Multi-spot

In [ ]:
url = 'http://files.figshare.com/1839122/12d_New_30p_320mW_steer_3.hdf5'
download_file(url, save_dir='./data')

In [ ]:
file_name = './data/' + url.split('/')[-1]
file_name

In [ ]:
d = loader.photon_hdf5(file_name)

In [ ]:
timestamps = d.ph_times_m[0]
acceptor_mask = d.A_em[0]

In [ ]:
d2 = Data(ph_times_m=[timestamps], A_em=[acceptor_mask], 
         clk_p=10e-9, ALEX=False, nch=1, fname='')

In [ ]:
d.calc_bg(bg.exp_fit, time_s=20, tail_min_us='auto', F_bg=1.7)

In [ ]:
dplot(d, timetrace_bg);

In [ ]:
d2.burst_search_t()

In [ ]:
dplot(d2, hist_fret)